In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skbio.stats.distance import DistanceMatrix
from skbio.diversity import beta_diversity
from skbio.stats.ordination import pcoa
from skbio import TreeNode
from biom import load_table
from scipy.spatial.distance import euclidean
from emperor import Emperor, nbinstall
nbinstall()
%matplotlib inline

In [2]:
data_dir = '../data'

otu_table = load_table('%s/deblur-clean-16s.biom' % data_dir)
mapping = pd.read_csv('%s/qiita_refined_mapping.csv' % data_dir, index_col=0)

ms_table = load_table('%s/MS1.biom' % data_dir)
ms_table = pd.DataFrame(np.array(ms_table.matrix_data.todense()).T,
                        index=ms_table.ids(axis='sample'),
                        columns=ms_table.ids(axis='observation'))

# OTU table

Here, we will use Unweighted Unifrac to calculate the distance matrix on samples
with respect to the OTU phylogeny

In [3]:
otu_table = pd.DataFrame(np.array(otu_table.matrix_data.todense()).T,
                         index=otu_table.ids(axis='sample'),
                         columns=otu_table.ids(axis='observation'))

In [4]:
otu_table = otu_table.astype(np.int)

In [5]:
hellinger = lambda x, y : euclidean(x, y) / np.sqrt(2) 

In [6]:
hellinger_dm = DistanceMatrix.from_iterable(otu_table.values, hellinger)
hellinger_dm.ids = otu_table.index

In [7]:
otu_pc = pcoa(hellinger_dm)

In [8]:
otu_floor_ids = mapping.loc[mapping.subject=='Floor'].index
mean_otu_floor = otu_pc.samples.loc[otu_floor_ids].mean(axis=0)
otu_floor_distance = otu_pc.samples.apply(lambda x: euclidean(x, mean_otu_floor), axis=1)

In [9]:
otu_pc.write('../results/otu_pcoa.pc')

'../results/otu_pcoa.pc'

# MS Bucket table

In [10]:
hellinger_dm = DistanceMatrix.from_iterable(ms_table.values, hellinger)
hellinger_dm.ids = ms_table.index

In [11]:
ms_pc = pcoa(hellinger_dm)

In [12]:
ms_floor_ids = mapping.loc[mapping.subject=='Floor'].index
mean_ms_floor = ms_pc.samples.loc[ms_floor_ids].mean(axis=0) 
ms_floor_distance = ms_pc.samples.apply(lambda x: euclidean(x, mean_ms_floor), axis=1)

In [13]:
ms_pc.write('../results/ms_pcoa.pc')

'../results/ms_pcoa.pc'

# Unleash the Emperor

In [14]:
#mapping.index = mapping['#SampleID']
mapping = pd.merge(mapping, pd.DataFrame({'otu_floor_distance': otu_floor_distance}),
                   left_index=True, right_index=True)
#mapping.index = mapping['ms_filename']
mapping = pd.merge(mapping, pd.DataFrame({'ms_floor_distance': ms_floor_distance}),
                   left_index=True, right_index=True)

mapping.to_csv('../data/dm_mapping.txt')

In [15]:
otu_samples = otu_pc.samples.loc[mapping.index]
otu_mapping = mapping.loc[[x in set(otu_samples.index) for x in mapping.index]]

ms_samples = ms_pc.samples.loc[mapping.index]
ms_mapping = mapping.loc[[x in set(ms_samples.index) for x in mapping.index]]

In [16]:
#otu_mapping.index = otu_mapping['#SampleID']
x = Emperor(otu_pc, otu_mapping, remote=True)

In [17]:
x

In [18]:
y = Emperor(ms_pc, ms_mapping, remote=True)

In [19]:
y

In [20]:
ms_mapping[['x', 'y', 'z', 'radius', 'ms_floor_distance']].to_csv('../results/ms_mapping_ili.csv')
otu_mapping[['x', 'y', 'z', 'radius', 'otu_floor_distance']].to_csv('../results/otu_mapping_ili.csv')

In [21]:
with open('../results/otu_emperor.html', 'w') as f:
    f.write(x.make_emperor(standalone=True))
    y.copy_support_files(target='../results/emperor')

In [22]:
with open('../results/ms_emperor.html', 'w') as f:
    f.write(y.make_emperor(standalone=True))
    y.copy_support_files(target='../results/emperor')